In [1]:
# dependences
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
import numpy as np

In [2]:
def process(img):
    # increase contrast
    # alpha = 1.5 # Contrast control (1.0-3.0)
    # beta = 0 # Brightness control (0-100)
    # img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    #process colours
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #gray_image = cv2.blur(gray_image,(3,3))
    
    #find edges
    edged = cv2.Canny(gray_image, 30, 200) 
    cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
    edged_image = img.copy()
    cv2.drawContours(edged_image,cnts,-1,(0,255,0),3)
    
    filtered_edged_image = img.copy()
    #filter contours

    min_ratio = 1000000
    min_ratio_cnt = 0
    real_ratio = 4.7
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)
        epsilon = 0.09*cv2.arcLength(c,True)
        approx = cv2.approxPolyDP(c,epsilon,True)
        aspect_ratio = float(w)/h
        
        ratio_diff = abs(aspect_ratio - real_ratio)
        if ratio_diff < min_ratio and len(approx)==4:
            min_ratio = ratio_diff
            min_ratio_cnt = c
    
    if type(min_ratio_cnt) != int:
        cv2.drawContours(filtered_edged_image,[min_ratio_cnt],-1,(0,255,0),3)
    return gray_image, edged_image,filtered_edged_image

In [ ]:
fig, axs = plt.subplots(len(os.listdir("fotos")), 2, sharex=True, sharey=True, figsize=(25, 150))

for i,filename in enumerate(os.listdir("fotos")):
    if filename[-3:] == "jpg":
        print(filename)
        image = cv2.imread('fotos/'+filename)
        gray_image, edged_image,filtered_edged_image = process(image)
        # try:
        #     Max = cnt.max(axis=0)
        #     #print(Max)
        #     cv2.circle(image_contours, Max, radius=100, color=(255,0,0), thickness=-1)
        #     Min = cnt.min(axis=0)
        #     cv2.circle(image_contours, Min, radius=100, color=(255,0,0), thickness=-1)
        # except:
        #     pass
        axs[i][0].imshow(edged_image)
        axs[i][1].imshow(filtered_edged_image)

# image = cv2.imread('fotos/'+"IMG_4283.jpg")
# image_contours,cnt = process(image)
# print(cnt.max(axis=0))
# print(cnt.min(axis=0))
# plt.imshow(image_contours)